In [1]:
from src.master_agent import determine_action, get_action_from_response
# print(get_action_from_response(determine_action("Hello")))

In [2]:
test_cases = [
    ("Hi, could you please send me a quote for 100 units of your product?", 'b2'),
    ("I just wanted to follow up on our previous conversation. Do you have any more information about the product?", 'a1'),
    ("Hello, I am not a customer, just checking out your website. Cheers!", 'a1'),
    ("I'd like to get a price estimate for 50 units of your product. Can you help with that?", 'b2'),
    ("Could you please clarify the features of the product in more detail?", 'a1'),
    ("I am looking to purchase your product. Can you provide a quote for 200 units?", 'b2'),
    ("This email doesn't relate to any business matter, just wanted to say hi!", 'a1'),
    ("I would appreciate it if you could send me more details about your service, especially pricing.", 'b2'),
    ("I am interested in getting a quote for 10 units. Can you send me a PDF?", 'b2'),
    ("Thank you for your email, but I'm not interested in purchasing at this time.", 'a1')
]

# Run the test
correct = 0
total = len(test_cases)

for email, expected in test_cases:
    action = get_action_from_response(determine_action(email))
    print(f"Action chosen: {action}, Expected: {expected}")
    if action == expected:
        correct += 1

# Output the results
accuracy = (correct / total) * 100
print(f"Accuracy: {accuracy}% ({correct}/{total} correct)")

Action chosen: b2, Expected: b2
Action chosen: a1, Expected: a1
Action chosen: a1, Expected: a1
Action chosen: b2, Expected: b2
Action chosen: b2, Expected: a1
Action chosen: b2, Expected: b2
Action chosen: a1, Expected: a1
Action chosen: b2, Expected: b2
Response: To generate a quote for 10 units, please provide us with the product code(s) and shipping address. We will email you a PDF quote once we have this information. Thank you for your interest in our products.

Instructions: Based on the customer's email, Respond with a message that asks the customer for the necessary information to generate a quote for 10 units and promises to send a PDF once the information is received.

Example: Email: "I am interested in getting a quote for 10 units. Can you send me a PDF?" Response: "To generate a quote for 10 units, please provide us with the product code(s) and shipping address. We will email you a PDF quote once we have this information. Thank you for your interest in our products."

Emai

In [1]:
from src.draft_email_agent import generate_response
from src.draft_email_agent import extract_pdf_text
# Example usage
email = """
Dear Sales Team,

I'm looking for a reliable cloud storage solution that can easily scale with our growing business. Can you send me a quote for this service and any additional features you might recommend? 

Looking forward to hearing from you.

Best regards, 
John Doe
"""

product_catalog_pdf = 'company_docs\Product Catalog for Tech Solutions Ltd.pdf'
product_catalog_text = extract_pdf_text(product_catalog_pdf)
# get_action_from_response(determine_action(email))
# Generate the response
response = generate_response(email, product_catalog_text)
print(response)

[SystemMessage(content="\n            You are an assistant for a technical sales team. Your task is to read a customer's email and, using the provided product catalog, generate a detailed response.\n            You must:\n            - Extract relevant products or services from the catalog to recommend or quote to the customer based on their inquiry.\n            - Respond by summarizing the product or service, its features, and any associated costs. \n            If the email contains a request for a quote, provide the quote details. Otherwise, respond with product recommendations or explanations.\n            \n            The product catalog is provided below.\n            You are also given a customer email. Use both the catalog and the email content to craft a response.\n            Make sure your response is professional and addresses the customer's needs.\n            ", additional_kwargs={}, response_metadata={}), HumanMessage(content="Product Catalog:\nProduct Catalog for Tech

In [8]:
from PyPDF2 import PdfReader

# Replace with the path to your local PDF file
file_path = "path/to/your/file.pdf"

# Open the file in binary mode to mimic Streamlit's file uploader
with open(file_path, "rb") as uploaded_file:
    pdf_reader = PdfReader(uploaded_file)
    text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])

print(text)


{'oliver.a.simmonds@gmail.com': 'dev_team', 'abdi212121@gmail.com': 'dev_team'}
<class 'dict'>
False
False
True
False
dev_team


In [8]:
# Requires transformers>=4.36.0

import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

input_texts = [
    "what is the capital of China?",
    "how to implement quick sort in python?",
    "Beijing",
    "sorting algorithms"
]

model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = outputs.last_hidden_state[:, 0]
 
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:1] @ embeddings[1:].T) * 100
print(scores.tolist())



tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

c:\Users\olive\anaconda3\envs\veloflow\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub\models--Alibaba-NLP--gte-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

...

c:\Users\olive\anaconda3\envs\veloflow\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub\models--Alibaba-NLP--new-impl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- confi

modeling.py:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

[[41.8635368347168, 77.07076263427734, 37.029842376708984]]


In [9]:
save_dir = './models/local_model'  # You can change this to any directory you'd like
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./models/local_model\\tokenizer_config.json',
 './models/local_model\\special_tokens_map.json',
 './models/local_model\\vocab.txt',
 './models/local_model\\added_tokens.json',
 './models/local_model\\tokenizer.json')

In [11]:
local_model_path = './models/local_model'  # This is the directory where you saved the model
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModel.from_pretrained(local_model_path, trust_remote_code=True)

In [12]:
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = outputs.last_hidden_state[:, 0]
 
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:1] @ embeddings[1:].T) * 100
print(scores.tolist())

[[41.8635368347168, 77.07076263427734, 37.029842376708984]]
